In [ ]:
using StatsBase

In [ ]:
struct Bairro
    nResidencias::Integer
    nPessoas::Integer
    residencias::Array{T} where T <: Integer
    residenciasPos::Array{T, 2} where T <: Number
    pessoas::Array{T} where T <: Integer
    distancias::Array{T} where T <: Number
end

struct Residencia
    n::Integer
    pessoas::Array{T} where T <: Integer
    bairro::Integer
    posicao::Array{T} where T <: Number
end

struct Social
    n::Integer
    pessoas::Array{T} where T <: Integer
end

struct Pessoas
    residencias::Array{T} where T <: Integer
    bairros::Array{T} where T <: Integer
    social::Array{T} where T <: Integer
    posicao::Array{T, 2} where T <: Number
end

struct Populacao
    n::Integer
    I0::Integer
    estadoInicial::Array{T} where T <: Integer
    estadoAtual::Array{T} where T <: Integer
    pessoas::Pessoas
    residencias::Array{Residencia}
    bairros::Array{Bairro}
    sociais::Array{Social}
end

In [ ]:
function selectiveRand(v)
    aux = zeros(length(v))
    aux[v] .= rand(sum(v))
    return aux
end

function rowWiseNorm(A)
    return sqrt.(sum(abs2, A, dims=2)[:, 1])
end

In [ ]:
function calculaDistancia(populacao::Populacao, suscetiveis, infectados, fKernel)
    aux = zeros(populacao.n)
    infectadosBairros = [sum(infectados[i.pessoas]) for i in populacao.bairros]
    for bairro in populacao.bairros
        suscetiveisBairro = bairro.pessoas[suscetiveis[bairro.pessoas]]
        infectadosBairro = bairro.pessoas[infectados[bairro.pessoas]]
        infectadosResidencia = [sum(infectados[i.pessoas]) for i in populacao.residencias[bairro.residencias]]
        aux2 = ones(length(suscetiveisBairro)) .* sum(infectadosBairros .* bairro.distancias)
        Threads.@threads for i in 1:length(suscetiveisBairro)
            aux2[i] += sum(fKernel(bairro.residenciasPos .- populacao.pessoas.posicao[suscetiveisBairro[i], :]') .* infectadosResidencia)
        end
        aux[suscetiveisBairro] .= aux2
    end
    return aux[suscetiveis]
end

function miniPassoMatricial(estados::Array{T} where T <: Integer)
    popSuscetiveis = estados.==1
    popInfectados = estados.==2
    
    n = length(estados)
    nInfectados = sum(popInfectados)
    nSuscetiveis = sum(popSuscetiveis)

    expostos = zeros(Bool, n, n)
    expostos[popSuscetiveis, popInfectados] .= true
    return sum(expostos, dims=2)[:, 1]
end

function passoMisto(populacao::Populacao, 
        α::Array{T} where T <: Number, β::Array{T} where T <: Number, θ::Array{T} where T <: Number, 
        γ::Number, δ::Number, fKernel)
    """
        Entrada:
            resumoPop: 
            α: taxa de transmissão residencial
            β: taxa de transmissão social
            θ: taxa de transmissão global
            γ: probabilidade de não recuperação
            δ: tamanho do passo temporal
            fKernel: ?
    """
    popSuscetiveis = populacao.estadoAtual.==1
    popInfectados = populacao.estadoAtual.==2
    
    contatos = zeros(populacao.n)
    Threads.@threads for i in populacao.residencias
        contatos[i.pessoas] .+= miniPassoMatricial(populacao.estadoAtual[i.pessoas]) .* α[i.pessoas]
    end
    
    Threads.@threads for i in populacao.sociais
        contatos[i.pessoas] .+= miniPassoMatricial(populacao.estadoAtual[i.pessoas]) .* β[i.pessoas]
    end
    
    contatos[popSuscetiveis] .+= calculaDistancia(populacao, popSuscetiveis, popInfectados, fKernel) .* θ[popSuscetiveis]
    
    prob = exp.(- δ .* contatos)
    
    novosInfectados = selectiveRand(popSuscetiveis) .> prob
    novosRecuperados = selectiveRand(popInfectados) .> γ
    
    infectados = ((popInfectados .& (.~novosRecuperados)) .| novosInfectados)
    suscetiveis = (popSuscetiveis .& (.~novosInfectados))

    return 3 .* ones(Int, populacao.n) .- 2 .* suscetiveis .- infectados
end

function evolucaoMista(
        populacao::Populacao, tempos::AbstractArray{T} where T <: Number, nSim::Integer, 
        α::Array{T} where T <: Number, β::Array{T} where T <: Number, θ::Array{T} where T <: Number, γ::Number, fKernel)
    """
        Entrada:
            resumoPop: 
            tempos:
            α: taxa de transmissão residencial
            β: taxa de transmissão social
            θ: taxa de transmissão global
            γ: parâmetro da exponencial de não recuperação
            δ: tamanho do passo temporal
            fKernel: ?
    """    
    nT = length(tempos)
    passos = tempos[2:end] - tempos[1:(end-1)]
    Γ = exp.(- γ .* passos)
    
    S = zeros(nSim, nT)
    I = zeros(nSim, nT)
    R = zeros(nSim, nT)
    
    S[:, 1] .= populacao.n - populacao.I0
    I[:, 1] .= populacao.I0
    
    for j in 1:nSim
        populacao.estadoAtual .= populacao.estadoInicial
        for (k, δ) in enumerate(passos)
            @time populacao.estadoAtual .= passoMisto(populacao, α, β, θ, Γ[k], δ, fKernel)
            
            S[j, k+1] = sum(populacao.estadoAtual .== 1)
            I[j, k+1] = sum(populacao.estadoAtual .== 2)
            R[j, k+1] = sum(populacao.estadoAtual .== 3)
        end
    end
    return S,I,R
end

In [ ]:
using BenchmarkTools

In [ ]:
function geraPopulacaoAleatoria(n, I0, nResidencias, nSociais, bairrosShape, fKernel)
    nBairros = prod(bairrosShape)
    pop0 = ones(Int, n)
    pop0[StatsBase.sample(1:n, I0, replace=false)] .= 2;

    residencia = rand(1:nResidencias, n)
    residencias = [Int[] for i in 1:nResidencias]   
    Threads.@threads for i in 1:nResidencias
        append!(residencias[i], (1:n)[residencia .== i])
    end
    residenciasPos = rand(nResidencias, 2) .* bairrosShape'
    residenciasBai = [
        ceil(Int, residenciasPos[i, 1]) + 
        floor(Int, residenciasPos[i, 2]) * bairrosShape[2]
        for i in 1:nResidencias
    ]
    residencias = [Residencia(length(j), j, residenciasBai[i], residenciasPos[i, :]) for (i, j) in enumerate(residencias)]
    
    social = rand(1:nSociais, n)
    sociais = [Social(sum(social .== i), (1:n)[social .== i]) for i in 1:nSociais]
    
    posicao = zeros(n, 2)
    bairro = zeros(Int, n)
    Threads.@threads for i in 1:n
        posicao[i, :] .= residenciasPos[residencia[i], :]
        bairro[i] = residenciasBai[residencia[i]]
    end
    
    pessoas = Pessoas(residencia, bairro, social, posicao)
    
    bairrosResidencias = [(1:nResidencias)[residenciasBai .== i] for i in 1:nBairros]
    bairrosPessoas = [(1:n)[bairro .== i] for i in 1:nBairros]
    centros = vcat([mean(posicao[i, :], dims=1) for i in bairrosPessoas]...)
    dist = hcat([fKernel(centros .- centros[i, :]') for i in 1:nBairros]...)
    replace!(dist, NaN=>0.)
    bairros = [
        Bairro(
            length(bairrosResidencias[i]),
            length(bairrosPessoas[i]),
            bairrosResidencias[i],
            residenciasPos[bairrosResidencias[i], :],
            bairrosPessoas[i],
            dist[i, :]
            ) for i in 1:nBairros
    ]
    
    return Populacao(n, I0, pop0, copy(pop0), pessoas, residencias, bairros, sociais)
end

In [ ]:
function powerDecay(a::Array)
    b = rowWiseNorm(a)
    return b ./ (b .+ 1.5);
end

In [ ]:
k = floor(Int, 1e+5 / 100)

n = 100 * k
I0 = 1 * k
nResidencias = 30 * k
nSociais = 1 * k
@time populacao = geraPopulacaoAleatoria(n, I0, nResidencias, nSociais, [2,2], powerDecay);

In [ ]:
using Plotly

In [ ]:
data = Dict([
    "x"=>populacao.pessoas.posicao[:, 1],
    "autobinx" => false,
    "xbins" => Dict([
        "start" => 0,
        "end" => 2,
        "size" => 1
    ]),
    "y"=>populacao.pessoas.posicao[:, 2],
    "autobiny" => false,
    "ybins" => Dict([
        "start" => 0,
        "end" => 1,
        "size" => 1
    ]),
]);

Plotly.plot(Plotly.histogram2d(data))

In [ ]:
data = Dict([
    "x"=>populacao.pessoas.posicao[:, 1],
    "y"=>populacao.pessoas.posicao[:, 2]
]);

Plotly.plot(Plotly.histogram2dcontour(data))

In [ ]:
α = 1e-1 * ones(n)
β = 1e-4 * ones(n)
@time θ = 1e-1 ./ calculaDistancia(populacao, ones(Bool, n), ones(Bool, n), powerDecay);
γ = 0.2;

In [ ]:
@time a = evolucaoMista(populacao, Array(1:10), 1, α, β, θ, γ, powerDecay)

In [ ]:
Base.summarysize(θ) / 1024 / 1024